<a href="https://colab.research.google.com/github/annamaartensson/dd2424project/blob/issue%2F2/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf

In [148]:
class RNN:

  class Grads:

    def __init__(self, m, K):
      self.b = tf.Variable(tf.zeros_initializer()(shape = (K)))
      self.c = tf.Variable(tf.zeros_initializer()(shape = (K)))
      self.U = tf.Variable(tf.zeros_initializer()(shape = (m, K)))
      self.W = tf.Variable(tf.zeros_initializer()(shape = (m, m)))
      self.V = tf.Variable(tf.zeros_initializer()(shape = (K, m)))

    def compute(self):
      pass

  def __init__(self, data, seq_length, m, K, sig):
    self.data = data
    self.seq_length = seq_length
    self.m = m
    self.K = K
    self.b = tf.Variable(tf.zeros_initializer()(shape = (m)))
    self.c = tf.Variable(tf.zeros_initializer()(shape = (K)))
    self.U = tf.Variable(tf.random_normal_initializer(mean=0.0, stddev=sig)(shape = (m, K)))
    self.W = tf.Variable(tf.random_normal_initializer(mean=0.0, stddev=sig)(shape = (m, m)))
    self.V = tf.Variable(tf.random_normal_initializer(mean=0.0, stddev=sig)(shape = (K, m)))
    self.hprev = tf.Variable(tf.zeros_initializer()(shape = (m)))

  def forwardPass(self, X):
    P = tf.Variable(tf.zeros_initializer()(shape = (self.K, self.seq_length)))
    H = tf.Variable(tf.zeros_initializer()(shape = (self.m, self.seq_length+1)))
    A = tf.Variable(tf.zeros_initializer()(shape = (self.m, self.seq_length)))
    H[:,0].assign(self.hprev)
    for t in range(self.seq_length):
      A[:,t].assign(tf.tensordot(self.W, H[:,t], 1) + tf.tensordot(self.U, X[:,t], 1) + self.b)
      H[:,t+1].assign(tf.math.tanh(A[:,t]))
      P[:,t].assign(tf.nn.softmax(tf.tensordot(self.V, H[:,t+1], 1) + self.c))
    return P, H, A

  def backwardPass(self, X, Y, P, H, A):
    grads = RNN.Grads(self.m, self.K)

    return grads

  def loss(self, Y, P):
    L = 0
    for t in range(self.seq_length):
        L -= tf.tensordot(Y[:,t], tf.math.log(P[:,t]), 1)
    return L

  def update(self, eta, grads):
    self.b.assign(self.b+eta*grads.b)
    self.c.assign(self.c+eta*grads.c)
    self.U.assign(self.U+eta*grads.U)
    self.W.assign(self.W+eta*grads.W)
    self.V.assign(self.V+eta*grads.V)

  def train(self, eta, steps):
    e = 0
    for iter in range(steps):
      X = self.data[:,e:e+self.seq_length]
      Y = self.data[:,e+1:e+self.seq_length+2]
      P, H, A = self.forwardPass(X)
      L = self.loss(Y, P)
      grads = self.backwardPass(X, Y, P, H, A)
      self.update(eta, grads)
      e = e+1
      #update hprev



In [142]:
path_to_file = tf.keras.utils.get_file('prideandprejudice.txt', 'https://www.gutenberg.org/cache/epub/42671/pg42671.txt')
text = list(open(path_to_file, 'rb').read().decode(encoding='utf-8'))

In [143]:
vocabulary = sorted(set(text))
char_to_ind = tf.keras.layers.StringLookup(vocabulary = list(vocabulary), mask_token = None)
ind_to_char = tf.keras.layers.StringLookup(vocabulary = char_to_ind.get_vocabulary(), invert = True, mask_token = None)

In [144]:
seq_length = 100
m = 100
K = len(vocabulary)
sig = 0.01
data = tf.transpose(tf.one_hot(char_to_ind(text), K))

In [149]:
rnn = RNN(data, seq_length, m, K, sig)
X = data[:,0:seq_length]
Y = data[:,1:seq_length+1]
P, H, A = rnn.forwardPass(X)
print(rnn.loss(Y, P))